In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import Window
import pandas as pd
import random

In [2]:
read_path = '/home/tatiane/Downloads/exemplo_linkage_cases-brazil-cities-time.csv/tmp/exemplo_linkage_cases-brazil-cities-time.csv'

In [3]:
df = spark.read.csv(read_path, sep = ",", multiLine = True)

In [4]:
#renomeando colunas
header = ["id1", "id2", "city1", "city2", "regiao_saude1", "regiao_saude2", "codmun1", "codmun2", "date1", "date2", "epi_week1", "epi_week2", "score"]

for nome_antigo, novo_nome in zip(df.columns, header):
    df = df.withColumnRenamed(nome_antigo, novo_nome)

In [5]:
df = df.withColumn('score', F.col('score').cast(DoubleType()))

In [6]:
# Descrição dos dados
df.select('score').summary().toPandas().set_index('summary') #.T #transforma coluna em linhas para facilitar a visão da tabela.

,score
summary,
count,2243800
mean,0.9604942705214136
stddev,0.07386957747356336
min,0.5165745856353592
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [7]:
df.select(F.substring('score', 1, 3).alias('faixa_score')).show()

+-----------+
|faixa_score|
+-----------+
|        1.0|
|        1.0|
|        1.0|
|        1.0|
|        0.7|
|        0.7|
|        0.7|
|        1.0|
|        0.7|
|        1.0|
|        1.0|
|        1.0|
|        0.7|
|        0.7|
|        0.7|
|        0.7|
|        1.0|
|        1.0|
|        0.7|
|        0.7|
+-----------+
only showing top 20 rows



In [8]:
df.groupBy(F.substring('score', 1, 4).alias('faixa_score')).count().show()

+-----------+-------+
|faixa_score|  count|
+-----------+-------+
|        1.0|1727048|
|       0.55|   2288|
|       0.68|     52|
|       0.82| 489104|
|       0.72|   8606|
|       null|   5177|
|       0.96|  11116|
|       0.51|      2|
|       0.65|     26|
|       0.93|   5558|
+-----------+-------+



In [9]:
# Distribuição dos dados
df.groupBy(F.round('score', 1)).count().show()

+---------------+-------+
|round(score, 1)|  count|
+---------------+-------+
|            0.7|   8684|
|           null|   5177|
|            1.0|1738164|
|            0.6|   2288|
|            0.8| 489104|
|            0.5|      2|
|            0.9|   5558|
+---------------+-------+



### Atividade: criar faixas de score
ex:
> 0.95 e <= 1 = 1 <br>
< 0.95 e >= 0.89 = 2 <br>
< 0.88 = 3

In [10]:
#df.withColumn('f_score', [(F.when(F.col('score') >= 0.95, 1).otherwise(0)), (F.when(F.col('score') >= 0.89 & F.when(F.col('score') < 0.95, 2).otherwise(0))), ])
#df.withColumn('f_score', F.when(F.col('score') >= 0.89 & F.when(F.col('score') < 0.95, 2).otherwise(0))
#df.withColumn('f_score', F.when(F.col('score') < 0.88, 3).otherwise(0))

Py4JError: An error occurred while calling o99.and. Trace:
py4j.Py4JException: Method and([class java.lang.Double]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [ ]:
df.withColumn(
    "fa_score",
    F.when(F.col('score') >= 0.95, 1).otherwise(0),
    F.when(F.col('score') >= 0.89 & F.when(F.col('score') < 0.95, 2).otherwise(0))
    F.when(F.col('score') < 0.88, 3).otherwise(0)
 )

In [ ]:
df.withColumn?